# Module 4. Custom Metric 으로 성능 데이터 및 Cold Start 성능 체크 하기
이번 모듈에서는 모듈2에서 테스트 용으로 분리했던 데이터를 가지고 Custom 지표를 통해 추가적인 성능을 평가해 보도록 합니다. 
또한 Coldstart 성능도 추가적으로 확인해 보도록 합니다.

In [1]:
import pandas as pd, numpy as np
import io
import scipy.sparse as ss
import json
import time
import os
import boto3
import uuid
from botocore.exceptions import ClientError
from metrics import mean_reciprocal_rank, ndcg_at_k, precision_at_k
!pip install tqdm
from tqdm import tqdm_notebook
from datetime import datetime
from random import randint

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [2]:
%store -r

In [3]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')
personalize_events = boto3.client('personalize-events')

In [4]:
df_all=pd.read_csv(interaction_all_file)
df_train=pd.read_csv(interaction_train_file)
df_test=pd.read_csv(interaction_test_file)

item_all=pd.read_csv(item_file)
item_cold=pd.read_csv(item_cold_file)
item_warm=pd.read_csv(item_warm_file)

In [5]:
#Unique users
unique_users=df_all['USER_ID'].unique()
print("unique_user_from_all dataset:", len(unique_users))

unique_user_from_train=df_train['USER_ID'].unique()
print("unique_user_from_train dataset:", len(unique_user_from_train))

unique_user_from_test=df_test['USER_ID'].unique()
print("unique_user_from_test dataset:", len(unique_user_from_test))

old_user_from_test=df_test['USER_ID'][df_test['USER_ID'].isin(unique_user_from_train)].unique()
print("Existing users in test dataset:",len(old_user_from_test))

new_user_from_test=df_test['USER_ID'][-df_test['USER_ID'].isin(unique_user_from_train)].unique()
print("New users in test dataset:",len(new_user_from_test))

cold_item_interaction=df_test[df_test['ITEM_ID'].isin(item_cold['ITEM_ID'].unique())]
cold_interaction_user=cold_item_interaction['USER_ID'].unique()
print("User with cold item interactions in test dataset:",len(cold_interaction_user))

old_user_from_test_with_cold=cold_item_interaction['USER_ID'][cold_item_interaction['USER_ID'].isin(old_user_from_test)].unique()
print("Exisiting User with cold item interaction in test dataset:", len(old_user_from_test_with_cold))

unique_user_from_all dataset: 6023
unique_user_from_train dataset: 4641
unique_user_from_test dataset: 1983
Existing users in test dataset: 601
New users in test dataset: 1382
User with cold item interactions in test dataset: 57
Exisiting User with cold item interaction in test dataset: 12



#### 조금더 상세하고 Custum 평가 지표를 얻기 위해서 이전에 분리해둔 테스트 데이터를 가지고 캠페인 생성 후 별도 테스트를 진행하도록 합니다.

In [6]:
# 테스트 인터렉션 셋에 있는 모든 사용자를 대상으로 선능을 확인해 봅니다.
relevance = []
counts=[]
for user_id in tqdm_notebook(unique_user_from_test):
    true_items = set(df_test[df_test['USER_ID']==user_id]['ITEM_ID'].values)
    #print(true_items)
  
    rec_response = personalize_runtime.get_recommendations(
            campaignArn =user_personalization_campaign_arn ,
            userId = str(user_id)
        )
    rec_items = [int(x['itemId']) for x in rec_response['itemList']] 
    relevance.append([int(x in true_items) for x in rec_items])

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  app.launch_new_instance()


In [7]:
print('mean_reciprocal_rank', np.mean([mean_reciprocal_rank(r) for r in relevance]))
print('precision_at_5', np.mean([precision_at_k(r, 5) for r in relevance]))
print('precision_at_10', np.mean([precision_at_k(r, 10) for r in relevance]))
print('precision_at_25', np.mean([precision_at_k(r, 25) for r in relevance]))
print('normalized_discounted_cumulative_gain_at_5', np.mean([ndcg_at_k(r, 5) for r in relevance]))
print('normalized_discounted_cumulative_gain_at_10', np.mean([ndcg_at_k(r, 10) for r in relevance]))
print('normalized_discounted_cumulative_gain_at_25', np.mean([ndcg_at_k(r, 25) for r in relevance]))

mean_reciprocal_rank 0.3944173097544351
precision_at_5 0.21321230458900658
precision_at_10 0.20342914775592538
precision_at_25 0.22642460917801316
normalized_discounted_cumulative_gain_at_5 0.2598970789183912
normalized_discounted_cumulative_gain_at_10 0.2904870526842617
normalized_discounted_cumulative_gain_at_25 0.4707628802541746


In [8]:
# 퍼스널라이즈에서 제공하는 메트릭과 비슷한지 비교해 봅니다.

get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = user_personalization_solution_version_arn
)

#print(json.dumps(get_solution_metrics_response, indent=2))

print('mean_reciprocal_rank',get_solution_metrics_response["metrics"]["mean_reciprocal_rank_at_25"], )
print('precision_at_5', get_solution_metrics_response["metrics"]["precision_at_5"])
print('precision_at_10', get_solution_metrics_response["metrics"]["precision_at_10"])
print('precision_at_25', get_solution_metrics_response["metrics"]["precision_at_25"])
print('normalized_discounted_cumulative_gain_at_5', get_solution_metrics_response["metrics"]["normalized_discounted_cumulative_gain_at_5"])
print('normalized_discounted_cumulative_gain_at_10', get_solution_metrics_response["metrics"]["normalized_discounted_cumulative_gain_at_10"])
print('normalized_discounted_cumulative_gain_at_25', get_solution_metrics_response["metrics"]["normalized_discounted_cumulative_gain_at_25"])

mean_reciprocal_rank 0.2426
precision_at_5 0.0993
precision_at_10 0.0916
precision_at_25 0.0767
normalized_discounted_cumulative_gain_at_5 0.1682
normalized_discounted_cumulative_gain_at_10 0.2283
normalized_discounted_cumulative_gain_at_25 0.3348


## 이벤트 트랙커 생성 

아래 코드 셀은 특정 item과 상호 작용하는 사용자를 시뮬레이트하는 코드 입니다. 이벤트 트레커를 통해 실시간 스트림을 보내도록 하여 고객의 클릭정보에 따라 추천 항목이 변하는 것을 확인할 수 있습니다. 

In [9]:
event_tracker_name="user-personalization-event-tracker-"+WORK_DATE
print(dataset_group_arn)
print(event_tracker_name)

arn:aws:personalize:ap-northeast-2:870180618679:dataset-group/20201215-dataset-group
user-personalization-event-tracker-20201215


In [10]:
event_tracker_response = personalize.create_event_tracker( 
    name=event_tracker_name,
    datasetGroupArn=dataset_group_arn
)
event_tracker_arn  = event_tracker_response['eventTrackerArn']
event_tracking_id = event_tracker_response['trackingId']

print('eventTrackerArn:{},\n eventTrackingId:{}'.format(event_tracker_arn, event_tracking_id))


eventTrackerArn:arn:aws:personalize:ap-northeast-2:870180618679:event-tracker/81e1bae6,
 eventTrackingId:401c2330-c611-4fb6-bfdf-2858747ca001


## 사용자 행동 시뮬레이션

아래 코드 셀은 특정 item과 상호 작용하는 사용자를 시뮬레이트하는 코드 샘플을 제공하며, 시작할 때와 다른 추천 목록을 얻습니다.

In [11]:
def get_movie_title(movie_id):
    """
    Takes in an ID, returns a title
    """
    movie_id = int(movie_id)
    movie_title=item_all[item_all['ITEM_ID']==movie_id]['TITLE']
    return (movie_title.tolist())

In [12]:
session_dict = {}

In [13]:
def send_movie_click(USER_ID, ITEM_ID):
    """
    Simulates a click as an envent
    to send an event to Amazon Personalize's Event Tracker
    """
    # Configure Session
    try:
        session_ID = session_dict[USER_ID]
    except:
        session_dict[USER_ID] = str(uuid.uuid1())
        session_ID = session_dict[USER_ID]
   
    value=randint(0,5)
    
    # Configure Properties:
    event = {
    "itemId": str(ITEM_ID),
    "eventValue": value
    }
    event_json = json.dumps(event)
    
    # Make Call
    personalize_events.put_events(
    trackingId = event_tracking_id, # 이벤트트래커에서 생성한 아이디
    userId= USER_ID,
    sessionId = session_ID,
    eventList = [{
        'sentAt': int(time.time()),
        'eventType': 'RATING',
        'properties': event_json
        }]
)
    

In [14]:
user_id=old_user_from_test[0]

get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = user_personalization_campaign_arn,
    userId = str(user_id),
)

item_list = get_recommendations_response['itemList']

recommendation_title_list = []
recommendation_id_list=[]
for item in item_list:
    title = get_movie_title(item['itemId'])
    recommendation_title_list.append(title)
    recommendation_id_list.append(item['itemId'])
recommendations_df = pd.DataFrame(recommendation_title_list ,columns = ['OriginalRecs'])
recommendations_df

,OriginalRecs
0,Braveheart (1995)
1,Star Wars: Episode VI - Return of the Jedi (1983)
2,Forrest Gump (1994)
3,One Flew Over the Cuckoo's Nest (1975)
4,Star Wars: Episode V - The Empire Strikes Back...
5,Star Wars: Episode I - The Phantom Menace (1999)
6,Star Wars: Episode IV - A New Hope (1977)
7,Schindler's List (1993)
8,Titanic (1997)
9,"Silence of the Lambs, The (1991)"


In [15]:
# Pick a movie, we will use ID 270 or Miracle on 34th Street

movie_to_click = item_warm['ITEM_ID'][0]
movie_title_clicked = get_movie_title(movie_to_click)
send_movie_click(USER_ID=str(user_id), ITEM_ID=movie_to_click)

In [16]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = user_personalization_campaign_arn,
    userId = str(user_id)
     
)

print("Recommendations for user: ", user_id)

item_list = get_recommendations_response['itemList']

recommendation_list = []

for item in item_list:
    title = get_movie_title(item['itemId'])
    recommendation_list.append(title)
    
new_rec_DF = pd.DataFrame(recommendation_list, columns = [movie_title_clicked])

recommendations_df = recommendations_df.join(new_rec_DF)
recommendations_df

Recommendations for user:  1389


,OriginalRecs,"(Toy Story (1995),)"
0,Braveheart (1995),Braveheart (1995)
1,Star Wars: Episode VI - Return of the Jedi (1983),Star Wars: Episode VI - Return of the Jedi (1983)
2,Forrest Gump (1994),Forrest Gump (1994)
3,One Flew Over the Cuckoo's Nest (1975),One Flew Over the Cuckoo's Nest (1975)
4,Star Wars: Episode V - The Empire Strikes Back...,Star Wars: Episode V - The Empire Strikes Back...
5,Star Wars: Episode I - The Phantom Menace (1999),Star Wars: Episode I - The Phantom Menace (1999)
6,Star Wars: Episode IV - A New Hope (1977),Star Wars: Episode IV - A New Hope (1977)
7,Schindler's List (1993),Schindler's List (1993)
8,Titanic (1997),Titanic (1997)
9,"Silence of the Lambs, The (1991)","Silence of the Lambs, The (1991)"


## Cold Item interaction 내보내기 

이번 실험에서는 콜드 스타트 아이템의 인터렉션을 발생하도록 합니다.

User-Personalization 레시피를 통해 솔루션을 생성했을경우 새로운 인터렉션이 있을때마다 아마존 퍼스널라이즈는 매 2시간 마다 솔루션 버전을 자동으로 업데이트 합니다. 이때 새로운 아이템이 포함 되어 있다면 설정해 놓은 Exploration Weight 및 Exploration item age cut off 값에 따라 콜드 아이템도 추천하게 됩니다. 자세한 내용은 [여기](https://docs.aws.amazon.com/personalize/latest/dg/recording-events.html)를 참고하세요. 

이 번 실습에서는 시간상 강제로 신규 아이템 인터렉션을 발생한 뒤 매뉴얼로 솔루션 버전 및 캠페인을 업데이트 하도록 합니다. 

In [66]:
new_item_list=item_cold
new_item_list['ITEM_ID']=new_item_list['ITEM_ID'].astype(str)
new_item_list

,ITEM_ID,TITLE,GENRE
0,136,From the Journals of Jean Seberg (1995),Documentary
1,138,"Neon Bible, The (1995)",Drama
2,192,"Show, The (1995)",Documentary
3,402,Open Season (1996),Comedy
4,576,Fausto (1993),Comedy
...,...,...,...
56,3542,Coming Apart (1969),Drama
57,3647,Running Free (2000),Drama
58,3772,Hatchet For the Honeymoon (Rosso Segno Della F...,Horror
59,3800,Criminal Lovers (Les Amants Criminels) (1999),Drama|Romance


In [69]:
user_id=old_user_from_test_with_cold[2]


In [75]:
for i in range(len(new_item_list)):
    value=randint(0,5)       
    # Configure Properties:
    event = {
    "itemId": str(new_item_list['ITEM_ID'][i]),
    "eventValue": value
    }
    event_json = json.dumps(event)
    
    
    
    personalize_events.put_events(
         trackingId = event_tracking_id,
         userId= str(user_id),
         sessionId = str(i),

         eventList = [{
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'RATINGS',
         'properties' : event_json
         }])

### 솔루션 버전 업데이트 및 캠페인 업데이트 

#### 이벤트 트레커 정보가 데이터 쌓일때가지 충분한 시간을 주기 위해 약 5분정도 후에 아래 셀을 시작합니다.

In [73]:
## 새로운 인터렉션을 포함한 새로운 솔루션 버전 생성 
create_solution_version_response = personalize.create_solution_version(
    solutionArn = user_personalization_solution_arn,
    trainingMode='UPDATE'
)

user_personalization_solution_version_arn_new = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:870180618679:solution/user-personalization-20201215/eafc56a3",
  "ResponseMetadata": {
    "RequestId": "41deaf46-ffd0-43b9-8ff6-1c86d4c552ef",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 15 Dec 2020 13:48:10 GMT",
      "x-amzn-requestid": "41deaf46-ffd0-43b9-8ff6-1c86d4c552ef",
      "content-length": "120",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [77]:
%%time

max_time = time.time() + 8*60*60 # 8 hours
while time.time() < max_time:
      
    #hrnn status
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = user_personalization_solution_version_arn_new
    )  
    status= describe_solution_version_response["solutionVersion"]["status"]
    print("User-Personalization SolutionVersion: {}".format(status))
       
    if  (status== "ACTIVE" or status == "CREATE FAILED"):
        break
 
    time.sleep(300)

print("All solution creation completed")

User-Personalization SolutionVersion: ACTIVE
All solution creation completed
CPU times: user 16.3 ms, sys: 2 µs, total: 16.3 ms
Wall time: 1.06 s



방금 업데이트된 새로운 솔루션 버전으로 캠페인을 업데이트 합니다. 이 때 explorationWeight를 1로 변경하여 최대한 콜드 아이템에서만 추천 될수 있도록 합니다. 

In [78]:
update_campaing_response = personalize.update_campaign(
    campaignArn=user_personalization_campaign_arn,
    solutionVersionArn=user_personalization_solution_version_arn_new,
    minProvisionedTPS=1,
    campaignConfig = {"itemExplorationConfig": {"explorationWeight": "1", "explorationItemAgeCutOff": "7"}}
    
)

user_personalization_campaign_arn = update_campaing_response['campaignArn']

print(json.dumps(update_campaing_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:ap-northeast-2:870180618679:campaign/user-personalization-campaign-20201215",
  "ResponseMetadata": {
    "RequestId": "24c10ccb-d229-450f-8b31-70d8c883e55e",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 15 Dec 2020 14:38:04 GMT",
      "x-amzn-requestid": "24c10ccb-d229-450f-8b31-70d8c883e55e",
      "content-length": "113",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [86]:
# Wait for campaign update to reflect the new explorationWeight
explorationWeight = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = user_personalization_campaign_arn
    )
    solution_version = describe_campaign_response["campaign"]["solutionVersionArn"]
    

    if (solution_version == user_personalization_solution_version_arn_new):
        print("Current Campaign solution version {}".format(solution_version))
        break
        
    time.sleep(60)


Current Campaign solution version arn:aws:personalize:ap-northeast-2:870180618679:solution/user-personalization-20201215/eafc56a3


## Cold Start Exploration Weight에 따른 테스트 

이부분에서는 새롭게 더해진 새로운 아이템(ColdStart)에 대한 추천 성능을 테스트 해보도록 합니다. 또한 Exploration Weight를 변경하면서 추천 아이템 중에 얼마 만큼의 콜드 아이템이 포함 되었는지 확인해 보도록 합니다.


In [87]:
metrics=[]

def build_metric_matrix(solution,relevance):
    metrics.append([solution,
                np.mean([mean_reciprocal_rank(r) for r in relevance]),
                np.mean([precision_at_k(r, 5) for r in relevance]),
                np.mean([precision_at_k(r, 10) for r in relevance]),
                np.mean([precision_at_k(r, 10) for r in relevance]),
                np.mean([ndcg_at_k(r, 5) for r in relevance]),
                np.mean([ndcg_at_k(r, 10) for r in relevance]),
                np.mean([ndcg_at_k(r, 25) for r in relevance]),
                np.mean(counts)])
    
def is_cold_item(rec_items):
    rec_is_cold=[int(item) for item in rec_items if int(item) in item_cold['ITEM_ID'].astype('int64').unique()]
                 #item_cold['ITEM_ID'].astype('int64')]
    return(rec_is_cold)


In [88]:
relevance = []
is_cold_item_list=[]
counts=[]
for user_id in tqdm_notebook(old_user_from_test):
    true_items = set(df_test[df_test['USER_ID']==user_id]['ITEM_ID'].values)
    #print(true_items)
  
    rec_response = personalize_runtime.get_recommendations(
            campaignArn =user_personalization_campaign_arn ,
            userId = str(user_id)
        )
    rec_items = [int(x['itemId']) for x in rec_response['itemList']]
    arr=is_cold_item(rec_items)  
    counts.append(len(arr))
    is_cold_item_list.append(arr)
    relevance.append([int(x in true_items) for x in rec_items])


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [89]:
print('mean_reciprocal_rank', np.mean([mean_reciprocal_rank(r) for r in relevance]))
print('precision_at_5', np.mean([precision_at_k(r, 5) for r in relevance]))
print('precision_at_10', np.mean([precision_at_k(r, 10) for r in relevance]))
print('precision_at_25', np.mean([precision_at_k(r, 25) for r in relevance]))
print('normalized_discounted_cumulative_gain_at_5', np.mean([ndcg_at_k(r, 5) for r in relevance]))
print('normalized_discounted_cumulative_gain_at_10', np.mean([ndcg_at_k(r, 10) for r in relevance]))
print('normalized_discounted_cumulative_gain_at_25', np.mean([ndcg_at_k(r, 25) for r in relevance]))
print('average number of cold items', np.mean(counts))
build_metric_matrix('user-personalization-coldstart-meta-update-100%',relevance)

mean_reciprocal_rank 0.09362662725586533
precision_at_5 0.03860232945091514
precision_at_10 0.03860232945091514
precision_at_25 0.04492512479201331
normalized_discounted_cumulative_gain_at_5 0.05720520806231752
normalized_discounted_cumulative_gain_at_10 0.07842399863787089
normalized_discounted_cumulative_gain_at_25 0.15459205991887345
average number of cold items 10.84026622296173


랜덤으로 추천하였을 경우 대비 Coldstart 성능이 얼마나 좋은 것인지 비교하여 보도록 합니다. 

In [90]:
relevance = []
for user_id in  tqdm_notebook(old_user_from_test):

    true_items = set(df_test[df_test['USER_ID']==user_id]['ITEM_ID'].values)
    rec_items = np.random.permutation(item_cold['ITEM_ID'].astype('int64').unique())[:25]
    relevance.append([int(x in true_items) for x in rec_items])
counts=[25]

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  from ipykernel import kernelapp as app


In [91]:

print('mean_reciprocal_rank', np.mean([mean_reciprocal_rank(r) for r in relevance]))
print('precision_at_5', np.mean([precision_at_k(r, 5) for r in relevance]))
print('precision_at_10', np.mean([precision_at_k(r, 10) for r in relevance]))
print('precision_at_25', np.mean([precision_at_k(r, 25) for r in relevance]))
print('normalized_discounted_cumulative_gain_at_5', np.mean([ndcg_at_k(r, 5) for r in relevance]))
print('normalized_discounted_cumulative_gain_at_10', np.mean([ndcg_at_k(r, 10) for r in relevance]))
print('normalized_discounted_cumulative_gain_at_25', np.mean([ndcg_at_k(r, 25) for r in relevance]))
print('average number of cold items', np.mean(counts))
build_metric_matrix('random',relevance)

mean_reciprocal_rank 0.0013430897998285684
precision_at_5 0.00033277870216306157
precision_at_10 0.00033277870216306157
precision_at_25 0.0006655574043261231
normalized_discounted_cumulative_gain_at_5 0.0010497999227478494
normalized_discounted_cumulative_gain_at_10 0.0016934817983460884
normalized_discounted_cumulative_gain_at_25 0.0038242318531639917
average number of cold items 25.0


### Update Campaign with 30% Exploration

In [92]:

update_campaing_response = personalize.update_campaign(
    campaignArn=user_personalization_campaign_arn,
    solutionVersionArn=user_personalization_solution_version_arn_new,
    minProvisionedTPS=1,
    campaignConfig = {"itemExplorationConfig": {"explorationWeight": "0.3", "explorationItemAgeCutOff": "7"}}
    
)

user_personalization_campaign_arn = update_campaing_response['campaignArn']

print(json.dumps(update_campaing_response, indent=2))


{
  "campaignArn": "arn:aws:personalize:ap-northeast-2:870180618679:campaign/user-personalization-campaign-20201215",
  "ResponseMetadata": {
    "RequestId": "fa1d3f83-a515-433a-92ab-2e8441b9282d",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 15 Dec 2020 15:16:45 GMT",
      "x-amzn-requestid": "fa1d3f83-a515-433a-92ab-2e8441b9282d",
      "content-length": "113",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [93]:
# Wait for campaign update to reflect the new explorationWeight
explorationWeight = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = user_personalization_campaign_arn
    )
    explorationWeight = describe_campaign_response["campaign"]["campaignConfig"]['itemExplorationConfig']['explorationWeight']
    print("Current Campaign  explorationWeight: {}".format(explorationWeight))
    
    if explorationWeight == "0.3":
        break
        
    time.sleep(60)


# wait 1 minutes
time.sleep(60)

Current Campaign  explorationWeight: 1
Current Campaign  explorationWeight: 0.3


In [94]:
relevance = []
is_cold_item_list=[]
counts=[]
for user_id in tqdm_notebook(old_user_from_test):
    true_items = set(df_test[df_test['USER_ID']==user_id]['ITEM_ID'].values)
    #print(true_items)
  
    rec_response = personalize_runtime.get_recommendations(
            campaignArn =user_personalization_campaign_arn ,
            userId = str(user_id)
        )
    rec_items = [int(x['itemId']) for x in rec_response['itemList']]
    arr=is_cold_item(rec_items)  
    counts.append(len(arr))
    is_cold_item_list.append(arr)
    relevance.append([int(x in true_items) for x in rec_items])

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [95]:
print('mean_reciprocal_rank', np.mean([mean_reciprocal_rank(r) for r in relevance]))
print('precision_at_5', np.mean([precision_at_k(r, 5) for r in relevance]))
print('precision_at_10', np.mean([precision_at_k(r, 10) for r in relevance]))
print('precision_at_25', np.mean([precision_at_k(r, 25) for r in relevance]))
print('normalized_discounted_cumulative_gain_at_5', np.mean([ndcg_at_k(r, 5) for r in relevance]))
print('normalized_discounted_cumulative_gain_at_10', np.mean([ndcg_at_k(r, 10) for r in relevance]))
print('normalized_discounted_cumulative_gain_at_25', np.mean([ndcg_at_k(r, 25) for r in relevance]))
print('average number of cold items', np.mean(counts))
build_metric_matrix('user-personalization-coldstart-30%',relevance)

mean_reciprocal_rank 0.09029524209663584
precision_at_5 0.03760399334442596
precision_at_10 0.036272878535773705
precision_at_25 0.0443261231281198
normalized_discounted_cumulative_gain_at_5 0.05443476885955114
normalized_discounted_cumulative_gain_at_10 0.07391097882485473
normalized_discounted_cumulative_gain_at_25 0.15053583759154687
average number of cold items 11.118136439267888


### Update Campaign with 0% Exploration

In [96]:

update_campaing_response = personalize.update_campaign(
    campaignArn=user_personalization_campaign_arn,
    solutionVersionArn=user_personalization_solution_version_arn_new,
    minProvisionedTPS=1,
    campaignConfig = {"itemExplorationConfig": {"explorationWeight": "0", "explorationItemAgeCutOff": "7"}}
    
)

user_personalization_campaign_arn = update_campaing_response['campaignArn']

print(json.dumps(update_campaing_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:ap-northeast-2:870180618679:campaign/user-personalization-campaign-20201215",
  "ResponseMetadata": {
    "RequestId": "89519f3f-f856-405e-ade4-18f73344eff9",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 15 Dec 2020 15:19:19 GMT",
      "x-amzn-requestid": "89519f3f-f856-405e-ade4-18f73344eff9",
      "content-length": "113",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [97]:
# Wait for campaign update to reflect the new explorationWeight
explorationWeight = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = user_personalization_campaign_arn
    )
    explorationWeight = describe_campaign_response["campaign"]["campaignConfig"]['itemExplorationConfig']['explorationWeight']
    print("Current Campaign  explorationWeight: {}".format(explorationWeight))
    
    if explorationWeight == "0":
        break
        
    time.sleep(60)

# wait 1 minutes
time.sleep(60)

Current Campaign  explorationWeight: 0.3
Current Campaign  explorationWeight: 0


In [98]:
relevance = []
is_cold_item_list=[]
counts=[]
for user_id in tqdm_notebook(old_user_from_test):
    true_items = set(df_test[df_test['USER_ID']==user_id]['ITEM_ID'].values)
    #print(true_items)
  
    rec_response = personalize_runtime.get_recommendations(
            campaignArn =user_personalization_campaign_arn ,
            userId = str(user_id)
        )
    rec_items = [int(x['itemId']) for x in rec_response['itemList']]
    arr=is_cold_item(rec_items)  
    counts.append(len(arr))
    is_cold_item_list.append(arr)
    relevance.append([int(x in true_items) for x in rec_items])

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [99]:
print('mean_reciprocal_rank', np.mean([mean_reciprocal_rank(r) for r in relevance]))
print('precision_at_5', np.mean([precision_at_k(r, 5) for r in relevance]))
print('precision_at_10', np.mean([precision_at_k(r, 10) for r in relevance]))
print('precision_at_25', np.mean([precision_at_k(r, 25) for r in relevance]))
print('normalized_discounted_cumulative_gain_at_5', np.mean([ndcg_at_k(r, 5) for r in relevance]))
print('normalized_discounted_cumulative_gain_at_10', np.mean([ndcg_at_k(r, 10) for r in relevance]))
print('normalized_discounted_cumulative_gain_at_25', np.mean([ndcg_at_k(r, 25) for r in relevance]))
print('average number of cold items', np.mean(counts))
build_metric_matrix('user-personalization-coldstart-0%',relevance)

mean_reciprocal_rank 0.18231356702163648
precision_at_5 0.08452579034941764
precision_at_10 0.08069883527454243
precision_at_25 0.0768053244592346
normalized_discounted_cumulative_gain_at_5 0.1245711493553611
normalized_discounted_cumulative_gain_at_10 0.15925591645458065
normalized_discounted_cumulative_gain_at_25 0.2510513274961014
average number of cold items 0.0


In [100]:

df_metrics=pd.DataFrame(metrics,columns=['recipe','mrr','p@5','p@10','p@25','ndcg@5','ndcg@10','ndcg@25','cold_item_count'])
df_metrics


,recipe,mrr,p@5,p@10,p@25,ndcg@5,ndcg@10,ndcg@25,cold_item_count
0,user-personalization-coldstart-meta-update-100%,0.093627,0.038602,0.038602,0.038602,0.057205,0.078424,0.154592,10.840266
1,random,0.001343,0.000333,0.000333,0.000333,0.001050,0.001693,0.003824,25.000000
2,user-personalization-coldstart-30%,0.090295,0.037604,0.036273,0.036273,0.054435,0.073911,0.150536,11.118136
3,user-personalization-coldstart-0%,0.182314,0.084526,0.080699,0.080699,0.124571,0.159256,0.251051,0.000000
